In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [2]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [3]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [ ]:
fitness_f

In [ ]:
behaviors = read_behaviors([files[6]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 358].head()

In [ ]:
behaviors.describe()

In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
b_grouped.index.get_values()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [4]:
fitness_data = read_moea_fitness([fitness_f[6]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [ ]:
hof = unpickle_moea('data/results/transferability_simulation_15/hof')

In [ ]:
len(hof)

In [ ]:
plot_moea_fitness(fitness_data, hof)

In [ ]:
for ind in hof:
    print(ind.key, ind.fitness.values)

In [5]:
genomes_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
transfered_file

In [ ]:
len(transfered_file)

In [6]:
g = sorted(genomes, key=lambda x: x.key)

In [7]:
t = sorted(transferred, key=lambda x: x.key)

In [ ]:
sim_genome = list(filter(lambda x: x.key == 398, g))[0]

In [ ]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time)

In [ ]:
trans_genome = list(filter(lambda x: x.key == 398, t))[0]

In [ ]:
print(trans_genome.key, trans_genome.fitness.values)

In [ ]:
sim_genome.sim_time

In [ ]:
plot_path([sim_genome, trans_genome])

In [ ]:
calc_str_disparity(trans_genome.position, sim_genome.position)

In [17]:
plot_surrogate_model(fitness_data)

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [8]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [18]:
str_disparities = [(s.gen, s.key, s.fitness.values[0], s.fitness.values[1], calc_str_disparity(t.position, s.position)) for s, t in union_t]    

In [19]:
str_disparities

[(1, 21, 0.2607287667366189, 0.11951712563775475, 0.21801469108551616),
 (1, 22, 0.37666657598379366, 0.11951712563775475, 0.17142255357559266),
 (1, 23, 0.14215099372303278, 0.11951712563775475, 0.10805629494983024),
 (1, 24, 0.21370346927401307, 0.11951712563775475, 0.8728595230875857),
 (1, 25, 0.4379590499598192, 0.11951712563775475, 0.3805276738600174),
 (1, 26, 1.15842073060853, 0.11951712563775475, 1.9957478844845054),
 (1, 27, 0.2798758363689432, 0.11951712563775475, 0.1555808565089477),
 (1, 28, 0.23028163405390123, 0.11951712563775475, 0.44598986478527974),
 (1, 29, 0.30669240212071647, 0.11951712563775475, 0.321640332580434),
 (1, 30, 0.051165632681825474, 0.11951712563775475, 0.08183377585456553),
 (1, 31, 0.22542595934786613, 0.11951712563775475, 0.13389847257463477),
 (1, 32, 0.2667900174542295, 0.11951712563775475, 0.23734452140100226),
 (1, 33, 0.010851058955115721, 0.11951712563775475, 0.07489888230277211),
 (1, 34, 0.0002714378104773918, 0.11951712563775475, 0.3684755

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [20]:
plot_str_disparity(str_disparities)

In [ ]:
np.arange(0, 11)

In [9]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [16]:
import scipy as sp
from scipy import stats

stats.pearsonr(actual, approximated)

(0.016407392036827017, 0.887373147485877)